In [0]:
!unzip -qq sf-crime.zip

In [2]:
!pip install PyDrive

    100% |████████████████████████████████| 993kB 11.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1eu6rMv94dQMH195VfnoztS68KfR-OnX4'})

In [0]:
download.GetContentFile('sf-crime.zip')

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [0]:
train_df = pd.read_csv('train.csv')

In [12]:
train_df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [18]:
train_df.Address.nunique()

23228

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
dayofweek = LabelEncoder()

In [0]:
pddistrict = LabelEncoder()

In [0]:
address = LabelEncoder()

In [0]:
train_df.DayOfWeek = dayofweek.fit_transform(train_df.DayOfWeek)

dummy_df = pd.get_dummies(train_df['Descript'])

In [0]:
train_df.PdDistrict = pddistrict.fit_transform(train_df.PdDistrict)

In [0]:
train_df.Address = address.fit_transform(train_df.Address)

In [30]:
train_df.head(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,37,WARRANT ARREST,6,4,"ARREST, BOOKED",19790,-122.425892,37.774599
1,2015-05-13 23:53:00,21,TRAFFIC VIOLATION ARREST,6,4,"ARREST, BOOKED",19790,-122.425892,37.774599


In [31]:
train_df.corr()

,Category,DayOfWeek,PdDistrict,Address,X,Y
Category,1.000000,0.000388,-0.040674,0.036355,-0.024401,-0.000414
DayOfWeek,0.000388,1.000000,0.009571,0.000344,-0.000279,0.000042
PdDistrict,-0.040674,0.009571,1.000000,-0.004098,-0.268083,0.016744
Address,0.036355,0.000344,-0.004098,1.000000,0.025043,0.015442
X,-0.024401,-0.000279,-0.268083,0.025043,1.000000,0.559338
Y,-0.000414,0.000042,0.016744,0.015442,0.559338,1.000000


In [0]:
category = LabelEncoder()

In [0]:
train_df.Category = category.fit_transform(train_df.Category)

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
log_x_train = train_df.iloc[:,[3,4,6,7,8]].values

In [0]:
random_classifier = RandomForestClassifier(n_estimators=10)

In [0]:
log_y_train = train_df.iloc[:,1].values

In [59]:
random_classifier.fit(log_x_train,log_y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [60]:
random_classifier.score(log_x_train,log_y_train)

0.42912183716398516

In [0]:
test_df = pd.read_csv('test.csv')

In [68]:
test_df.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y,Category
0,0,2015-05-10 23:59:00,3,0,6407,-122.399588,37.735051,16
1,1,2015-05-10 23:51:00,3,0,9744,-122.391523,37.732432,21
2,2,2015-05-10 23:50:00,3,4,6336,-122.426002,37.792212,27
3,3,2015-05-10 23:45:00,3,2,10633,-122.437394,37.721412,1
4,4,2015-05-10 23:45:00,3,2,10633,-122.437394,37.721412,1


In [0]:
test_df.DayOfWeek = dayofweek.transform(test_df.DayOfWeek)

In [16]:
test_df.Address.nunique()

23184

In [0]:
test_df.PdDistrict = pddistrict.transform(test_df.PdDistrict)

In [0]:
test_df.Address = address.fit_transform(test_df.Address)

In [0]:
x_test = test_df.iloc[:,[2,3,4,5,6]].values

In [0]:
rand_pred = random_classifier.predict(x_test)

In [0]:
naive_pred=naive_pred.reshape(-1,1)

In [0]:
test_df['Category'] = rand_pred

In [0]:
cat_df = pd.DataFrame()

In [0]:
cat_df['Id'] = test_df['Id']

In [75]:
cat_df.head()

,Id
0,0
1,1
2,2
3,3
4,4


In [0]:
cat_dummy = pd.get_dummies(test_df['Category'])

In [0]:
cols=pd.get_dummies(test_df['Category']).columns.astype(int)

In [0]:
namecols = category.inverse_transform(cols)

In [0]:
cat_dummy.columns = namecols

In [97]:
cat_dummy.head()

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS,TREA
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
dummy_train = pd.read_csv('train.csv')

In [0]:
reqcols=pd.get_dummies(dummy_train['Category']).columns

In [0]:
remcol =[]
for _ in reqcols:
  if not _ in namecols:
    remcol.append(_)

In [95]:
remcol

['TREA']

In [0]:
cat_dummy['TREA']=0

In [0]:
submission_df=cat_df.join(cat_dummy)

In [103]:
submission_df.shape

(884262, 40)

In [0]:
submission_df.to_csv('submission.csv',index=False)